# Inverse QSAR: activity-conditioned structure-from-activity framework

Цель: не универсальное предсказание MIC, а выявление структурных/физико-химических детерминант низкого MIC (высокого pMIC) на `train_lit` (строки 0–89), с отдельной проверкой правил на `experimental_holdout` (строки 90+).

In [ ]:

# Environment, reproducibility, logging
from pathlib import Path
import json
import logging
import random
import sys
import warnings

SEED = 42
random.seed(SEED)

ROOT = Path.cwd()
DATA_PATH = ROOT / "potok.csv"
ART = ROOT / "artifacts"
FIG = ART / "figures"
ART.mkdir(parents=True, exist_ok=True)
FIG.mkdir(parents=True, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler(ART / "run.log", mode="w", encoding="utf-8"), logging.StreamHandler(sys.stdout)],
)
logger = logging.getLogger("inverse_qsar")

config = {
    "seed": SEED,
    "task": "inverse_qsar_structure_from_activity",
    "split": "rows 0-89 -> train_lit; rows 90+ -> experimental_holdout",
    "low_mic_method": "Q25 of MIC on train_lit",
    "cv": "GroupKFold by Murcko scaffold on train_lit only",
    "morgan": {"radius": 2, "nBits": 1024},
    "y_random_n": 50,
}
(ART / "run_config.json").write_text(json.dumps(config, indent=2, ensure_ascii=False), encoding="utf-8")
logger.info("Config saved")


In [ ]:

# Dependency check (safe stop, no pip install inside notebook)
try:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats
    from sklearn.base import clone
    from sklearn.decomposition import PCA
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_selection import mutual_info_regression
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import (
        roc_auc_score, average_precision_score, balanced_accuracy_score,
        precision_score, recall_score, roc_curve, precision_recall_curve
    )
    from sklearn.model_selection import GroupKFold
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.tree import DecisionTreeClassifier

    from rdkit import Chem
    from rdkit.Chem import AllChem, Descriptors
    from rdkit.Chem.MolStandardize import rdMolStandardize
    from rdkit.Chem.Scaffolds import MurckoScaffold
except Exception as e:
    msg = (
        "Требуется локальная среда с RDKit + numpy/pandas/scipy/scikit-learn/matplotlib. "
        "Установка через pip в ноутбуке отключена для воспроизводимости."
    )
    logger.error(msg)
    logger.error(f"Import error: {e}")
    raise SystemExit(msg)

np.random.seed(SEED)
warnings.filterwarnings("ignore")


## Data split and target engineering

In [ ]:

if not DATA_PATH.exists():
    raise FileNotFoundError(DATA_PATH)

raw = pd.read_csv(DATA_PATH)
raw.columns = [c.strip() for c in raw.columns]

smiles_col = next((c for c in raw.columns if c.lower() in ["smiles","smile","canonical_smiles"]), None)
mic_col = next((c for c in raw.columns if c.lower() in ["mic","activity","target","y"]), None)
if smiles_col is None or mic_col is None:
    raise ValueError("Need SMILES and MIC-like column")

df = raw[[smiles_col, mic_col]].rename(columns={smiles_col: "smiles", mic_col: "MIC"}).copy()
df["row_id"] = np.arange(len(df))
df["MIC"] = pd.to_numeric(df["MIC"], errors="coerce")

train_lit = df.iloc[:90].copy()
experimental_holdout = df.iloc[90:].copy() if len(df) >= 90 else df.iloc[0:0].copy()
if len(df) < 90:
    logger.warning("<90 rows: experimental_holdout is empty")

# Target engineering only from train_lit
q25_mic = float(train_lit["MIC"].quantile(0.25))
train_lit["pMIC"] = -np.log10(train_lit["MIC"])
train_lit["low_MIC"] = (train_lit["MIC"] <= q25_mic).astype(int)

experimental_holdout["pMIC"] = -np.log10(experimental_holdout["MIC"]) if len(experimental_holdout) else []
experimental_holdout["low_MIC"] = (experimental_holdout["MIC"] <= q25_mic).astype(int) if len(experimental_holdout) else []

logger.info(f"Rows: total={len(df)}, train_lit={len(train_lit)}, experimental_holdout={len(experimental_holdout)}")
logger.info(f"low_MIC threshold (Q25 MIC on train_lit) = {q25_mic:.6g}")


## Curation and feature generation

In [ ]:

lfc = rdMolStandardize.LargestFragmentChooser()

def std_smiles(smi):
    m = Chem.MolFromSmiles(str(smi))
    if m is None:
        return None
    m = lfc.choose(m)
    if m is None:
        return None
    return Chem.MolToSmiles(m, canonical=True)


def descs(m):
    return {
        "MolWt": Descriptors.MolWt(m),
        "MolLogP": Descriptors.MolLogP(m),
        "TPSA": Descriptors.TPSA(m),
        "HBD": Descriptors.NumHDonors(m),
        "HBA": Descriptors.NumHAcceptors(m),
        "RotB": Descriptors.NumRotatableBonds(m),
        "RingCount": Descriptors.RingCount(m),
        "HeavyAtomCount": Descriptors.HeavyAtomCount(m),
    }


def scaffold(smi):
    m = Chem.MolFromSmiles(smi)
    s = MurckoScaffold.MurckoScaffoldSmiles(mol=m)
    return s if s else "ACYCLIC"


def build_features(block, block_name):
    rows, fps, reports, bit_infos = [], [], [], []
    for _, r in block.iterrows():
        s = std_smiles(r["smiles"])
        if s is None or pd.isna(r["MIC"]) or r["MIC"] <= 0:
            reports.append({"block": block_name, "row_id": int(r["row_id"]), "original_smiles": r["smiles"], "standardized_smiles": s, "MIC": r["MIC"], "action": "drop"})
            continue
        m = Chem.MolFromSmiles(s)
        d = descs(m)
        d.update({"smiles": s, "MIC": float(r["MIC"]), "pMIC": float(-np.log10(r["MIC"])), "low_MIC": int(r["MIC"] <= q25_mic), "scaffold": scaffold(s), "row_id": int(r["row_id"])})
        rows.append(d)
        bi = {}
        fp = AllChem.GetMorganFingerprintAsBitVect(m, radius=config["morgan"]["radius"], nBits=config["morgan"]["nBits"], bitInfo=bi)
        fps.append(np.array(fp, dtype=int))
        bit_infos.append(bi)
        reports.append({"block": block_name, "row_id": int(r["row_id"]), "original_smiles": r["smiles"], "standardized_smiles": s, "MIC": r["MIC"], "action": "keep"})
    feat = pd.DataFrame(rows)
    fp_df = pd.DataFrame(fps, columns=[f"bit_{i}" for i in range(config["morgan"]["nBits"])]) if len(fps) else pd.DataFrame()
    rep = pd.DataFrame(reports)
    return feat, fp_df, rep, bit_infos

train_feat, train_fp, rep1, train_bitinfo = build_features(train_lit, "train_lit")
exp_feat, exp_fp, rep2, exp_bitinfo = build_features(experimental_holdout, "experimental_holdout")
pd.concat([rep1, rep2], ignore_index=True).to_csv(ART / "curation_report.csv", index=False)

num_desc = [c for c in train_feat.columns if c in ["MolWt","MolLogP","TPSA","HBD","HBA","RotB","RingCount","HeavyAtomCount"]]
assert train_feat["smiles"].isna().sum() == 0
assert (train_feat["MIC"] > 0).all()


## Step 2: Continuous SAR correlations (train_lit only)

In [ ]:

corr_rows = []
for c in num_desc:
    sp = stats.spearmanr(train_feat[c], train_feat["pMIC"], nan_policy="omit")
    kd = stats.kendalltau(train_feat[c], train_feat["pMIC"], nan_policy="omit")
    mi = mutual_info_regression(train_feat[[c]], train_feat["pMIC"], random_state=SEED)[0]
    corr_rows.append({
        "descriptor": c,
        "spearman_rho": sp.correlation,
        "spearman_p": sp.pvalue,
        "kendall_tau": kd.correlation,
        "kendall_p": kd.pvalue,
        "mutual_information": mi,
    })
cor = pd.DataFrame(corr_rows).sort_values("spearman_p").reset_index(drop=True)
m = len(cor)
rank = np.arange(1, m+1)
cor["fdr_bh_q"] = np.minimum.accumulate((cor["spearman_p"].values * m / rank)[::-1])[::-1]
cor.to_csv(ART / "descriptor_correlations.csv", index=False)


## Step 3: Univariate active-vs-inactive statistics

In [ ]:

def cliffs_delta(x, y):
    gt = sum(1 for xi in x for yi in y if xi > yi)
    lt = sum(1 for xi in x for yi in y if xi < yi)
    return (gt - lt) / (len(x) * len(y))

rows = []
y_bin = train_feat["low_MIC"].values
for c in num_desc:
    x = train_feat[c].values
    x1 = x[y_bin==1]
    x0 = x[y_bin==0]
    if len(x1) < 3 or len(x0) < 3:
        continue
    u = stats.mannwhitneyu(x1, x0, alternative="two-sided")
    d = cliffs_delta(x1, x0)
    auc = roc_auc_score(y_bin, x)
    fpr, tpr, thr = roc_curve(y_bin, x)
    j = tpr - fpr
    idx = int(np.argmax(j))
    best_thr = float(thr[idx])
    sens, spec = float(tpr[idx]), float(1-fpr[idx])

    lr = LogisticRegression(penalty='l2', solver='liblinear', random_state=SEED)
    lr.fit(x.reshape(-1,1), y_bin)
    beta = float(lr.coef_[0][0])
    # approximate SE from Hessian
    p = lr.predict_proba(x.reshape(-1,1))[:,1]
    W = np.diag(p*(1-p))
    Xd = np.c_[np.ones(len(x)), x]
    cov = np.linalg.pinv(Xd.T @ W @ Xd)
    se = float(np.sqrt(max(cov[1,1], 1e-12)))
    orr = float(np.exp(beta))
    lcl = float(np.exp(beta - 1.96*se))
    ucl = float(np.exp(beta + 1.96*se))

    rows.append({
        "descriptor": c,
        "mannwhitney_u": u.statistic,
        "p_value": u.pvalue,
        "cliffs_delta": d,
        "roc_auc": auc,
        "youden_threshold": best_thr,
        "sensitivity": sens,
        "specificity": spec,
        "odds_ratio": orr,
        "or_ci_low": lcl,
        "or_ci_high": ucl,
    })
uni = pd.DataFrame(rows).sort_values("p_value")
uni.to_csv(ART / "univariate_statistics.csv", index=False)


## Step 4: Scaffold-aware classification of low_MIC

In [ ]:

X_desc = train_feat[num_desc].copy()
X_fp = train_fp.copy()
y = train_feat["low_MIC"].values
groups = train_feat["scaffold"].values

models = {
    "logreg_l2": Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(penalty="l2", solver="liblinear", random_state=SEED, max_iter=5000))]),
    "logreg_l1": Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(penalty="l1", solver="liblinear", random_state=SEED, max_iter=5000))]),
    "rf": RandomForestClassifier(n_estimators=400, random_state=SEED, n_jobs=-1),
}

cv = GroupKFold(n_splits=min(5, len(np.unique(groups))))
metrics_rows = []
roc_plot = {}
pr_plot = {}

for name, mdl in models.items():
    fold_scores = {k: [] for k in ["roc_auc","pr_auc","balanced_accuracy","precision","recall"]}
    y_all, p_all = [], []
    for tr, te in cv.split(X_desc, y, groups):
        Xtr = X_fp.iloc[tr] if name=="rf" else X_desc.iloc[tr]
        Xte = X_fp.iloc[te] if name=="rf" else X_desc.iloc[te]
        m = clone(mdl)
        m.fit(Xtr, y[tr])
        p = m.predict_proba(Xte)[:,1]
        pred = (p >= 0.5).astype(int)
        fold_scores["roc_auc"].append(roc_auc_score(y[te], p))
        fold_scores["pr_auc"].append(average_precision_score(y[te], p))
        fold_scores["balanced_accuracy"].append(balanced_accuracy_score(y[te], pred))
        fold_scores["precision"].append(precision_score(y[te], pred, zero_division=0))
        fold_scores["recall"].append(recall_score(y[te], pred, zero_division=0))
        y_all.extend(y[te]); p_all.extend(p)

    y_all = np.array(y_all); p_all = np.array(p_all)
    fpr, tpr, _ = roc_curve(y_all, p_all)
    rec, prec, _ = precision_recall_curve(y_all, p_all)
    roc_plot[name] = (fpr, tpr)
    pr_plot[name] = (rec, prec)

    metrics_rows.append({
        "model": name,
        "roc_auc_mean": np.mean(fold_scores["roc_auc"]), "roc_auc_std": np.std(fold_scores["roc_auc"]),
        "pr_auc_mean": np.mean(fold_scores["pr_auc"]), "pr_auc_std": np.std(fold_scores["pr_auc"]),
        "balanced_accuracy_mean": np.mean(fold_scores["balanced_accuracy"]),
        "precision_mean": np.mean(fold_scores["precision"]),
        "recall_mean": np.mean(fold_scores["recall"]),
    })

cls = pd.DataFrame(metrics_rows).sort_values("roc_auc_mean", ascending=False)
cls.to_csv(ART / "classification_metrics.csv", index=False)


## Step 5: Fragment enrichment (Morgan bits)

In [ ]:

rows = []
y = train_feat["low_MIC"].values
for bit in train_fp.columns:
    pres = train_fp[bit].values.astype(int)
    a = int(np.sum((pres==1) & (y==1)))
    b = int(np.sum((pres==1) & (y==0)))
    c = int(np.sum((pres==0) & (y==1)))
    d = int(np.sum((pres==0) & (y==0)))
    table = np.array([[a,b],[c,d]])
    odds, p = stats.fisher_exact(table)
    # corrected OR and EF
    t = table + 0.5
    or_corr = (t[0,0]*t[1,1])/(t[0,1]*t[1,0])
    p_act = (a + c) / max(a+b+c+d,1)
    p_bit_act = a / max(a+b,1)
    ef = p_bit_act / max(p_act,1e-12)
    rows.append({"bit": bit, "a":a, "b":b, "c":c, "d":d, "odds_ratio":odds, "odds_ratio_corrected":or_corr, "fisher_p":p, "enrichment_factor":ef})
frag = pd.DataFrame(rows).sort_values("fisher_p").reset_index(drop=True)
m = len(frag)
frag["fdr_bh_q"] = np.minimum.accumulate((frag["fisher_p"].values * m / (np.arange(1,m+1)))[::-1])[::-1]
frag.to_csv(ART / "fragments_enrichment.csv", index=False)


## Step 6: Rule extraction (shallow tree)

In [ ]:

top_desc = uni.sort_values("p_value").head(4)["descriptor"].tolist()
top_bits = frag.sort_values("fisher_p").head(6)["bit"].tolist()
rule_features = top_desc + top_bits
X_rule = pd.concat([train_feat[top_desc], train_fp[top_bits]], axis=1)
y = train_feat["low_MIC"].values

tree = DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, random_state=SEED)
tree.fit(X_rule, y)

leaf_id = tree.apply(X_rule)
rules = []
for leaf in np.unique(leaf_id):
    idx = np.where(leaf_id == leaf)[0]
    support = len(idx)
    if support == 0:
        continue
    pred_pos = np.mean(y[idx])
    pred = 1 if pred_pos >= 0.5 else 0
    if pred != 1:
        continue
    precision = np.mean(y[idx]==1)
    recall = np.sum(y[idx]==1)/max(np.sum(y==1),1)
    lift = precision / max(np.mean(y==1),1e-12)
    rules.append({"rule_id": int(leaf), "rule_text": f"Leaf {leaf} (tree-derived rule)", "precision": precision, "recall": recall, "support": support, "lift": lift})

rules_df = pd.DataFrame(rules).sort_values(["lift","precision"], ascending=False)
rules_df.to_csv(ART / "rules.csv", index=False)


## Step 7: Check rules/windows on experimental_holdout (90+)

In [ ]:

checks = []
if len(exp_feat):
    # descriptor windows from Youden thresholds and direction by AUC
    uni_idx = uni.set_index("descriptor")
    top_desc_use = [d for d in top_desc if d in uni_idx.index]
    top_bits_use = [b for b in top_bits if b in exp_fp.columns]

    X_exp_rule = pd.concat([exp_feat[top_desc_use], exp_fp[top_bits_use]], axis=1)
    exp_leaf = tree.apply(X_exp_rule.reindex(columns=X_rule.columns, fill_value=0))
    rule_hit = np.isin(exp_leaf, rules_df["rule_id"].values) if len(rules_df) else np.zeros(len(exp_feat), dtype=bool)

    for i in range(len(exp_feat)):
        desc_hits = {}
        for d in top_desc_use:
            thr = uni_idx.loc[d, "youden_threshold"]
            direction_high = uni_idx.loc[d, "roc_auc"] >= 0.5
            ok = exp_feat.iloc[i][d] >= thr if direction_high else exp_feat.iloc[i][d] <= thr
            desc_hits[d] = bool(ok)

        frag_hits = {b: bool(exp_fp.iloc[i][b] == 1) for b in top_bits_use}
        checks.append({
            "smiles": exp_feat.iloc[i]["smiles"],
            "row_id": int(exp_feat.iloc[i]["row_id"]),
            "matches_descriptor_windows": all(desc_hits.values()) if len(desc_hits) else False,
            "contains_top_fragment": any(frag_hits.values()) if len(frag_hits) else False,
            "matches_rule": bool(rule_hit[i]),
            "low_MIC_observed": int(exp_feat.iloc[i]["low_MIC"]),
        })

exp_check = pd.DataFrame(checks)
exp_check.to_csv(ART / "experimental_rule_check.csv", index=False)


## Step 8: Y-randomization for classification

In [ ]:

best_model_name = cls.iloc[0]["model"]
best_model = models[best_model_name]
X_best = X_fp if best_model_name=="rf" else X_desc

rng = np.random.default_rng(SEED)
rows = []
for i in range(config["y_random_n"]):
    ys = rng.permutation(train_feat["low_MIC"].values)
    aucs = []
    for tr, te in cv.split(X_best, ys, groups):
        m = clone(best_model)
        m.fit(X_best.iloc[tr], ys[tr])
        p = m.predict_proba(X_best.iloc[te])[:,1]
        aucs.append(roc_auc_score(ys[te], p))
    rows.append({"iter": i+1, "roc_auc_mean": float(np.mean(aucs))})

yrnd = pd.DataFrame(rows)
yrnd.to_csv(ART / "y_random_classification.csv", index=False)

plt.figure(figsize=(6,4))
plt.hist(yrnd["roc_auc_mean"], bins=15, alpha=0.8, label="y-scrambled")
plt.axvline(float(cls.iloc[0]["roc_auc_mean"]), color='r', ls='--', label=f"real {best_model_name}")
plt.xlabel("ROC-AUC (GroupKFold mean)")
plt.ylabel("Count")
plt.title("Y-randomization for classification")
plt.legend()
plt.tight_layout()
plt.savefig(ART / "y_random_plot.png", dpi=220)
plt.close()


## Step 9: Visualizations

In [ ]:

# ROC curves
plt.figure(figsize=(6,5))
for name, (fpr,tpr) in roc_plot.items():
    plt.plot(fpr, tpr, label=name)
plt.plot([0,1],[0,1],'k--')
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title("ROC curves (OOF aggregated)")
plt.legend(); plt.tight_layout(); plt.savefig(FIG / "roc_curves.png", dpi=220); plt.close()

# PR curves
plt.figure(figsize=(6,5))
for name, (rec,prec) in pr_plot.items():
    plt.plot(rec, prec, label=name)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("PR curves (OOF aggregated)")
plt.legend(); plt.tight_layout(); plt.savefig(FIG / "pr_curves.png", dpi=220); plt.close()

# Boxplots for top descriptors
box_desc = top_desc if len(top_desc) else num_desc[:4]
for d in box_desc:
    plt.figure(figsize=(5,4))
    sns.boxplot(data=train_feat, x="low_MIC", y=d)
    plt.xlabel("low_MIC (1=low MIC)")
    plt.ylabel(d)
    plt.title(f"{d}: low_MIC vs others")
    plt.tight_layout(); plt.savefig(FIG / f"boxplot_{d}.png", dpi=220); plt.close()

# Top fragments barplot
top_frag = frag.sort_values("fisher_p").head(15)
plt.figure(figsize=(8,4))
plt.bar(top_frag["bit"], top_frag["enrichment_factor"])
plt.xticks(rotation=90)
plt.ylabel("Enrichment factor")
plt.title("Top fragment bits enrichment")
plt.tight_layout(); plt.savefig(FIG / "top_fragments_enrichment.png", dpi=220); plt.close()

# PCA (train active/inactive + experimental)
train_X = train_feat[num_desc].copy(); train_lbl = np.where(train_feat["low_MIC"]==1, "train_low_MIC", "train_other")
if len(exp_feat):
    exp_X = exp_feat[num_desc].copy(); exp_lbl = np.array(["experimental"]*len(exp_feat))
    X_all = pd.concat([train_X, exp_X], ignore_index=True)
    lbl = np.concatenate([train_lbl, exp_lbl])
else:
    X_all = train_X; lbl = train_lbl

Xp = StandardScaler().fit_transform(X_all)
pcs = PCA(n_components=2, random_state=SEED).fit_transform(Xp)
pca_df = pd.DataFrame({"PC1": pcs[:,0], "PC2": pcs[:,1], "set": lbl})
plt.figure(figsize=(6,5))
for name, g in pca_df.groupby("set"):
    plt.scatter(g["PC1"], g["PC2"], label=name, alpha=0.8)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA: activity groups and experimental")
plt.legend(); plt.tight_layout(); plt.savefig(FIG / "pca_activity_experimental.png", dpi=220); plt.close()


## Step 10: Statistical framing and defense outputs

In [ ]:

best = cls.iloc[0]
summary = [
    "Concept: activity-conditioned structural analysis (inverse QSAR / structure-from-activity).",
    f"low_MIC threshold defined on train_lit only: MIC <= Q25 = {q25_mic:.6g}.",
    f"Best scaffold-aware classifier: {best['model']} with ROC-AUC={best['roc_auc_mean']:.3f}±{best['roc_auc_std']:.3f} and PR-AUC={best['pr_auc_mean']:.3f}±{best['pr_auc_std']:.3f}.",
    "Statistical evidence: descriptor correlations (Spearman/Kendall/MI), univariate tests (Mann-Whitney, Cliff’s delta, OR), fragment enrichment (Fisher + FDR), and rule extraction.",
    "Experimental holdout used only for post-hoc rule/window checks; never for training, thresholding, or CV.",
    "Limitations: small sample size, source bias, experimental variability, scaffold imbalance.",
]
(ART / "statistical_summary.txt").write_text("\n".join(summary), encoding="utf-8")

speech = [
    "Мы используем inverse QSAR: не пытаемся предсказать MIC для всех структур,",
    "а выделяем детерминанты низкого MIC через статистику и scaffold-aware проверку.",
    "Порог low_MIC задан строго по train_lit (нижний квартиль MIC),",
    "после чего подтверждаем вклад дескрипторов и фрагментов независимыми тестами.",
    "Experimental 90+ не участвует в обучении и служит только для проверки правил.",
    "Итог — rule-based framework для дизайна, с честными ограничениями по размеру и смещению данных.",
]
(ART / "defense_speech.txt").write_text("\n".join(speech), encoding="utf-8")


## Final notes

- `train_lit`: строки 0–89 `potok.csv`.
- `experimental_holdout`: строки 90+; не участвуют в обучении, выборе порогов и CV.
- Формулировки о "predictive QSAR" намеренно убраны в пользу activity-conditioned statistical framework.